In [ ]:
#Installing Library- PyMuPDF: used to convert pdf to html
!pip install PyMuPDF

     |████████████████████████████████| 6.3MB 5.0MB/s 


In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import pickle
import fitz
import re
import statistics 
from statistics import mode

In [ ]:
#This function is used when main headings, headings and subheadings can be distinctly classified with the bold tag
def getFontBold(page, start, end):
  fontSizes = []
  for i in range(start, end):
    index = page[i]

    #getText("html"): used to extract the html of that particular page
    tempHTMLString = index.getText("html")

    #Used to find the location of img tags: for 5 occurences, imgOccuraanceList holds five positions
    imgOccuranceList = [i.start() for i in re.finditer('<img', tempHTMLString)]
    for i in range(len(imgOccuranceList)): #Replace all img tags with '<>' 
      tempHTMLString = tempHTMLString.replace(tempHTMLString[tempHTMLString.find('<img'): tempHTMLString.find('>',tempHTMLString.find('<img')) + 1], '<>')

    #Extract font-size for each line
    for line in tempHTMLString.splitlines():
      if (line.find('<b>') != -1): #To operate on the line, only if the bold tag exists 
        individualFont = line[line.find('font-size:') + 10: line.find('pt',line.find('font-size:'))] #To extract the digits placed such as: 'font-size: 10pt'
        individualFont = float(individualFont) #Convert the str to float
        fontSizes.append(individualFont) #Append to list

  fontSizes = list(set(fontSizes))#Getting rid of multiple occurences
  fontSizes.sort() #Sorting the list
  return fontSizes

In [ ]:
#This function is used when main headings, headings and subheadings cannot be distinctly classified with the bold tag
def getFont(page, start, end):
  fontSizes = []
  for i in range(start, end):
    index = page[i]

    #getText("html"): used to extract the html of that particular page
    tempHTMLString = index.getText("html")

    #Used to find the location of img tags: for 5 occurences, imgOccuraanceList holds five positions
    imgOccuranceList = [i.start() for i in re.finditer('<img', tempHTMLString)]
    for i in range(len(imgOccuranceList)): #Replace all img tags with '<>' 
      tempHTMLString = tempHTMLString.replace(tempHTMLString[tempHTMLString.find('<img'): tempHTMLString.find('>',tempHTMLString.find('<img')) + 1], '<>')

    #Extract font-size for each line  
    for line in tempHTMLString.splitlines():
      if (line.find('div') == -1 and line.find('<>') == -1): #To operate on the line, only if div tag and '<>' doesnt exist
        individualFont = line[line.find('font-size:') + 10: line.find('pt',line.find('font-size:'))] #To extract the digits placed such as: 'font-size: 10pt'
        individualFont = float(individualFont) #Convert the str to float
        fontSizes.append(individualFont) #Append to list

  mostFrequentFont = mode(fontSizes) #Since all the lines are taken into consideration, most frequently occuring font-size is the size of the common text.
  fontSizes = list(set(fontSizes)) #Getting rid of multiple occurences
  fontSizes.sort() #Sorting the list
  fontSizes = [i for i in fontSizes if i >= mostFrequentFont] #To save only those fonts that are greater than or equal to the most frequent font, since main headings, headings and subheadings must always have font-size greater than text font size.
  return fontSizes

In [ ]:
#To determine the main headings, headings and subheadings according to the font sizes returned from the above two functions.
def getLabels(page, start, end):

  #Use either getFont or getFontBold according to PDF
  lt=getFont(page,start,end)

  finalList = []  
  for i in range(start, end):
    individualFont = 0
    index = page[i]

    #getText("html"): used to extract the html of that particular page
    tempHTMLString = index.getText("html")

    #Used to find the location of img tags: for 5 occurences, imgOccuraanceList holds five positions
    imgOccuranceList = [i.start() for i in re.finditer('<img', tempHTMLString)]
    for i in range(len(imgOccuranceList)): #Replace all img tags with '<>' 
      tempHTMLString = tempHTMLString.replace(tempHTMLString[tempHTMLString.find('<img'): tempHTMLString.find('>',tempHTMLString.find('<img')) + 1], '<>')


    for line in tempHTMLString.splitlines():
      try:
        if line.find('div') != -1:  #Ignore the div tag
          continue
        
        if (line.find('div') == -1 and line.find('<>') == -1): #To operate on the line, only if div tag and '<>' doesnt exist
          individualText = line[line.find('>',line.find("<span ")) + 1: line.find('</span>')] #To extract text placed in between span tags
          individualFont = line[line.find('font-size:') + 10: line.find('pt',line.find('font-size:'))] #To extract the digits placed such as: 'font-size: 10pt'
          styling = re.sub(line[line.find('>',line.find("<span ")) + 1: line.find('</span>')],'',line) #To extract the HTML styling

          individualFont = float(individualFont) #Convert the str to float

          #The following if conditions have been made after manually looking at a particular pdf and matching the font sizes to those of the main headings, headings and subheadings
          if ( individualFont == lt[len(lt) - 1]): #Font size extracted matches the largest font size: Main Heading
            print('Main Heading: ' + individualText)
            individualFont = 1 #Label: 1
            finalList.append([individualText,styling,individualFont]) #Text, HTML styling, Label
          else:
            if ( individualFont == lt[len(lt) - 2]): #Font size extracted matches the second-largest font size: Heading
              print('Heading: ' + individualText)
              individualFont = 2 #Label: 2
              finalList.append([individualText,styling,individualFont]) #Text, HTML styling, Label
            else:
              if ( individualFont == lt[len(lt) - 3]): #Font size extracted matches the third-largest font size: Sub Heading
                if (line.find('<b>') != -1):
                  print('Sub Heading: ' + individualText)
                  individualFont = 3 #Label: 3
                  finalList.append([individualText,styling,individualFont]) #Text, HTML styling, Label
              else:
                if ( individualFont == lt[len(lt) - 4]): #Font size extracted matches the fourth-largest font size: Sub Sub Heading
                  print('Sub Sub Heading: ' + individualText)
                  individualFont = 4 #Label: 4
                  finalList.append([individualText,styling,individualFont]) #Text, HTML styling, Label
                else:
                  if ( styling[styling.find('font-family:') + 12: styling.find(';',styling.find('font-family:'))] == "UbuntuMono,monospace"): #Font size extracted matches 'UbuntuMono,monospace': Code
                    print('Code ' + individualText)
                    individualFont = 6 #Label: 6
                    finalList.append([individualText,styling,individualFont]) #Text, HTML styling, Label
      except:
        continue    
  return finalList